In [16]:
from selenium import webdriver
# from msedge.selenium_tools import Edge, EdgeOptions
import chromedriver_binary
import csv

In [17]:
#chrome
driver = webdriver.Chrome()
#Firefox
# driver = webdriver.Firefox()
#MSEdge
# options = EdgeOptions()
# options.use_chromium = True
# driver = Edge(options=options)

In [18]:
url = 'https://www.amazon.com/'
driver.get(url)

In [19]:
def my_url(keyword):
    temp = 'https://www.amazon.com/s?k={}'
    keyword = keyword.replace(' ', '+')
    return temp.format(keyword)

In [20]:
url = my_url('laptop charger')

In [21]:
url

'https://www.amazon.com/s?k=laptop+charger'

In [22]:
driver.get(url)

In [23]:
from bs4 import BeautifulSoup

bs = BeautifulSoup(driver.page_source, 'html.parser')

In [24]:
def change_page(page):
    temp = url 
    temp += '&page={}'.format(page)
    return url

In [25]:
bs_result = bs.findAll('div', {'data-component-type': 's-search-result'})

In [32]:
bs_result[0].h2.a

<a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" href="/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&amp;adId=A00815132YZXS43M3V1MC&amp;qualifier=1672840205&amp;id=8695903683728305&amp;widgetName=sp_atf&amp;url=%2FCertification-Universal-Compatible-Ultrabook-Chromebook%2Fdp%2FB09NM28SB4%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dlaptop%2Bcharger%26qid%3D1672840205%26sr%3D8-1-spons%26psc%3D1"><span class="a-size-medium a-color-base a-text-normal">90W ETL Certification Universal AC Adapter Laptop Charger Compatible with Dell HP Acer Asus Lenovo IBM Toshiba Samsung Sony Fujitsu Gateway Notebook Ultrabook Chromebook Power Supply Cord with 16 Tips</span> </a>

In [ ]:
def extract_records(obj):
    atag = obj.h2.a
    des = obj.h2.a.text.strip()
    url = 'https://www.amazon.com/' + atag.get('href')
    try:
        parent = obj.find('span', 'a-price')
        price = parent.find('span', 'a-offscreen').text
    except AttributeError:
        return 
    try:
        rate = obj.find('span', 'a-icon-alt')